In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataset_path = "/content/drive/MyDrive/Dog_heart_project/Dog_heart"
test_path = "/content/drive/MyDrive/Dog_heart_project/Test"


In [4]:
import os

# List subfolders in Train and Valid
print("Train Classes:", os.listdir(f"{dataset_path}/Train"))
print("Valid Classes:", os.listdir(f"{dataset_path}/Valid"))

# List sample images from each class
print("Sample images from Train/Normal:", os.listdir(f"{dataset_path}/Train/Normal")[:5])
print("Sample images from Train/Small:", os.listdir(f"{dataset_path}/Train/Small")[:5])


Train Classes: ['Small', 'Normal', 'Large']
Valid Classes: ['Large', 'Small', 'Normal']
Sample images from Train/Normal: ['303.png', '305.png', '306.png', '313.png', '316.png']
Sample images from Train/Small: ['307.png', '325.png', '326.png', '108.png', '119.png']


In [23]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image

# Force images to RGB
def pil_loader_rgb(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

# Define transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load datasets
train_dataset = ImageFolder(root=f"{dataset_path}/Train", transform=transform, loader=pil_loader_rgb)
valid_dataset = ImageFolder(root=f"{dataset_path}/Valid", transform=transform, loader=pil_loader_rgb)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

print("Classes:", train_dataset.class_to_idx)


Classes: {'Large': 0, 'Normal': 1, 'Small': 2}


In [28]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3)
        )

    def forward(self, x):
        return self.network(x)


In [29]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ImprovedCNN().to(device)  # <-- Move model after device is defined

optimizer = optim.Adam(model.parameters(), lr=0.00005, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()


In [30]:
def train_model(model, train_loader, valid_loader, epochs=60):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, valid_loader, epochs=60)


Epoch 1/60, Loss: 1.0860
Epoch 2/60, Loss: 1.0190
Epoch 3/60, Loss: 0.9779
Epoch 4/60, Loss: 0.9723
Epoch 5/60, Loss: 0.9591
Epoch 6/60, Loss: 0.9489
Epoch 7/60, Loss: 0.9373
Epoch 8/60, Loss: 0.9271
Epoch 9/60, Loss: 0.9083
Epoch 10/60, Loss: 0.8967
Epoch 11/60, Loss: 0.8632
Epoch 12/60, Loss: 0.8610
Epoch 13/60, Loss: 0.8386
Epoch 14/60, Loss: 0.8259
Epoch 15/60, Loss: 0.8128
Epoch 16/60, Loss: 0.7898
Epoch 17/60, Loss: 0.7952
Epoch 18/60, Loss: 0.7751
Epoch 19/60, Loss: 0.7639
Epoch 20/60, Loss: 0.7638
Epoch 21/60, Loss: 0.7345
Epoch 22/60, Loss: 0.7470
Epoch 23/60, Loss: 0.7203
Epoch 24/60, Loss: 0.7387
Epoch 25/60, Loss: 0.7158
Epoch 26/60, Loss: 0.7032
Epoch 27/60, Loss: 0.7240
Epoch 28/60, Loss: 0.7136
Epoch 29/60, Loss: 0.6882
Epoch 30/60, Loss: 0.6834
Epoch 31/60, Loss: 0.6873
Epoch 32/60, Loss: 0.6759
Epoch 33/60, Loss: 0.6857
Epoch 34/60, Loss: 0.6977
Epoch 35/60, Loss: 0.6681
Epoch 36/60, Loss: 0.6681
Epoch 37/60, Loss: 0.6620
Epoch 38/60, Loss: 0.6678
Epoch 39/60, Loss: 0.

In [31]:
def evaluate_model(model, valid_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Final Model Accuracy: {(100 * correct / total):.2f}%")

evaluate_model(model, valid_loader)


Final Model Accuracy: 71.00%


In [32]:
from torchvision import transforms

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [33]:
# Use official 400 image names expected by MATLAB checker
allowed_filenames = [
    "1621.png", "1622.png", "81_12_1.png", "94.png", "100.png",  # ... etc (400 total)
    # Paste the rest of the filenames here from professor's list
]


In [37]:
with open("/content/clean_allowed_filenames.txt", "r") as f:
    allowed_filenames = [line.strip() for line in f if line.strip()]

print("Filtered filenames:", len(allowed_filenames))  # Should print 400


Filtered filenames: 400


In [38]:
import os
import pandas as pd
from PIL import Image

# Define test transform (no augmentation)
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Filter and sort test filenames
test_filenames = sorted([f for f in os.listdir(test_path) if f in allowed_filenames])

def predict_selected_images(model, test_path, transform, filenames):
    model.eval()
    predictions = []
    for img_name in filenames:
        image = Image.open(os.path.join(test_path, img_name)).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(image)
            _, predicted = torch.max(output, 1)
        predictions.append((img_name, predicted.item()))
    return predictions

# Generate predictions
test_predictions = predict_selected_images(model, test_path, test_transform, test_filenames)

# Save to CSV
df = pd.DataFrame(test_predictions, columns=["filename", "label"])
df.to_csv("/content/test_predictions_final_400.csv", index=False)

print("CSV saved as test_predictions_final_400.csv")


CSV saved as test_predictions_final_400.csv
